In [4]:
from google.cloud import bigquery
from google.cloud.bigquery import job
import pandas as pd
import os
from tqdm import tqdm
import pandas as pd
PROJCECT = 'ballosodeuk'
bq = bigquery.Client(project=PROJCECT)
import statsmodels.api as sm 
import numpy as np

In [60]:
# 클라이언트 설정
client = bigquery.Client()
query_name = "1017_쿠폰구매데이터"
# SQL 파일 읽기
# file = './query/쿠팡0403_0509.sql'
file = f'../query/{query_name}.sql'
with open(file, 'r') as file:
    query = file.read()


In [61]:
job_config = bigquery.QueryJobConfig()
query_job = client.query(query, job_config=job_config)

In [62]:
df = query_job.to_dataframe()

c:\Users\Owner\miniconda3\envs\pymc\lib\site-packages\google\cloud\bigquery\table.py:2309: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
c:\Users\Owner\miniconda3\envs\pymc\lib\site-packages\google\cloud\bigquery\table.py:2323: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
c:\Users\Owner\miniconda3\envs\pymc\lib\site-packages\google\cloud\bigquery\table.py:2337: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)


In [63]:
df.to_parquet('./0503_1022_earningSpending.parquet')

In [64]:
df = pd.read_parquet('./0503_1022_earningSpending.parquet')

In [65]:
df.rename(columns={'9000-12999':'9000-10999'},inplace=True)
df.entropy.mean()


0.3487963132403713

In [7]:
df.columns

Index(['User_ID', 'Mean_Spend_Ratio', 'entropy', '0-1000', '1000-2999',
       '3000-4999', '5000-6999', '7000-8999', '9000-10999', '11000 - 12999',
       '13000 - 14999', '15000 - 16999', '17000 - 18999', '19000 - 20999',
       '21000 - 22999', '23000 - 24999', '25000 - 26999', '27000 - 28999',
       '29000 - 30999', '31000 - 32999', '33000 - 34999', '35000 - 36999',
       '37000 - 38999', '39000 - 40999', '41000 -'],
      dtype='object')

- 통계치

In [70]:
def ci_stats(df,col):
    mean = df[col].mean()
    std = df[col].std()
    n = len(df)
    ci_upper = mean + 1.96 * std / np.sqrt(n)
    ci_lower = mean - 1.96 * std / np.sqrt(n)
    
    return round(mean,2), round(ci_upper,2), round(ci_lower,2)

spend_ratio = ci_stats(df,'Mean_Spend_Ratio')
entropy = ci_stats(df,'entropy')
Interval_Days = ci_stats(df,'Interval_Days')

print(spend_ratio, entropy, Interval_Days)

(0.68, 0.68, 0.68) (0.35, 0.35, 0.35) (10.02, 10.1, 9.94)


In [68]:
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler()
tst = df[['Mean_Spend_Ratio','entropy']]

tst_scaled = sc.fit_transform(tst)
pd.DataFrame(tst_scaled).corr()

,0,1
0,1.000000,-0.073605
1,-0.073605,1.000000


In [69]:
df

,User_ID,Mean_Spend_Ratio,Interval_Days,entropy,0-1000,1000-2999,3000-4999,5000-6999,7000-8999,9000-10999,...,23000 - 24999,25000 - 26999,27000 - 28999,29000 - 30999,31000 - 32999,33000 - 34999,35000 - 36999,37000 - 38999,39000 - 40999,41000 -
0,39867749-164c-43cc-8433-4cc0daa17956,0.875367,11.00,0.00,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,26e03ea0-f7d0-43a3-a00c-0268d48d66f1,0.000000,0.00,0.00,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1fe76913-af99-4177-98d4-1f6b0bef1dbf,0.694106,17.75,1.38,0,0,0,4,2,1,...,0,0,0,0,0,0,0,0,0,0
3,23cc23d1-e38a-4d11-854d-e005a88bd41f,1.000000,72.50,1.00,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,060764f0-51cf-468f-9029-84729dca69e8,0.890869,0.00,0.00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163933,1059813e-9bd6-45ae-ba23-572bd604a39c,0.973611,0.00,0.00,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
163934,ff7556e4-88e7-4e2c-8247-f3084bc7d5f4,0.933672,0.00,0.00,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
163935,d6103d4a-bd38-44e5-9ca1-bff3fc628fb3,0.451592,0.00,0.00,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
163936,84cfd59a-0b64-4b01-87e4-2c024d2b648b,0.626040,0.00,0.00,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


### 소비 강도의 증가에 따른 구매금액 범주 속할 확률 증가분

#### 1만원 이하 구매자의 경우, 탕진 확률이 낮다. 이상의 경우, 탕진 확률이 높다


#### 9월 소량 데이터
- print(np.exp(low_coef)) #1.1만원 이하 -> 강도 증가 시 0.14 배 낮아짐
- print(np.exp(low_mid)) #2.1만원 이하 -> 강도 증가 시 2.15배 높아짐
- print(np.exp(mid)) #3.1만원 이하 -> 강도 증가 시 12배 높아짐
- print(np.exp(high)) #4.1 만 이하, 이상 -> 강도 증가 시 30배 높아짐

#### 5~10월 전체 (16만명)
- 1.1만원 이하 : 강도 증가 시 0.04배 낮아짐
- 2.1만원 이하 : 강도 증가 시 1.78배 높아짐
- 3.1만원 이하 : 강도 증가 시 3.88배 높아짐
- 4.1만원 이하 : 강도 증가 시 15.4배 높아짐

>1. 따라서, 1.1만원 이하 구매의 경우 한탕 소진이 아니다. <br>
>2. 이 이상 높아질 시, 한탕 소진으로 모든 금액을 소진할 확률이 높다.
>3. 큰 쿠폰 구매자는 돈을 모아서 도달하고, 소액 쿠폰 구매는 잔액을 거의 대부분 남겨놓는 경향이 있다.
>4. 이 금액이 쇼핑지원금 타겟이다

In [71]:
def pre_prep(df):
    df['low'] = df[['0-1000', '1000-2999', '3000-4999', '5000-6999','7000-8999', '9000-10999', ]].sum(axis=1)
    df['low-mid'] = df[['11000 - 12999', '13000 - 14999', '15000 - 16999', '17000 - 18999', '19000 - 20999']].sum(axis=1)
    df['mid'] = df[['21000 - 22999', '23000 - 24999', '25000 - 26999', '27000 - 28999','29000 - 30999']].sum(axis=1)
    df['high'] = df[['31000 - 32999', '33000 - 34999', '35000 - 36999', '37000 - 38999', '39000 - 40999', '41000 -']].sum(axis=1)

    df_ = df[['User_ID','Mean_Spend_Ratio','low','low-mid','mid','high']].copy()
    df_ = df_[df_['Mean_Spend_Ratio'].notna()]
    df_ = df_.dropna(subset=['Mean_Spend_Ratio'])  # 결측값 제거
    df_ = df_[~df_['Mean_Spend_Ratio'].isin([float('inf'), -float('inf')])] 

    return df_

In [72]:
def logit_func(df_):
    X = df_['Mean_Spend_Ratio']
    y = df_.iloc[:,2:]
    y['low'] = y['low'].apply(lambda x : 1 if x > 1 else x)
    y['low-mid'] = y['low-mid'].apply(lambda x : 1 if x > 1 else x)
    y['mid'] = y['mid'].apply(lambda x : 1 if x > 1 else x)
    y['high'] = y['high'].apply(lambda x : 1 if x > 1 else x)

    results = {}
    for column in y.columns:
        y_column = y[column]
        X_with_const = sm.add_constant(X)
        model = sm.Logit(y_column, X_with_const).fit()
        results[column] = model.summary()

    return results


In [73]:
import pandas as pd
import numpy as np

def get_results(results):
    extracted_results = {}
    for column, summary in results.items():
        # summary.tables[1]에서 계수와 p-value 추출
        # 첫 번째 행은 상수항이므로, 두 번째 행부터 시작
        for i in range(1, len(summary.tables[1].data)):  # 첫 번째 행(상수항)을 제외하고 반복
            coef = summary.tables[1].data[i][1]  # 계수
            p_value = summary.tables[1].data[i][4]  # p-value
            variable_name = summary.tables[1].data[i][0]  # 변수 이름
            extracted_results[f"{column}_{variable_name}"] = {'coef': coef, 'p-value': p_value}        

    # DataFrame 생성
    df = pd.DataFrame(extracted_results).T

    # odds-ratio 계산
    def safe_exp(x):
        try:
            # x를 float로 변환
            float_x = float(x)
            return np.exp(float_x)  # exp 적용
        except (ValueError, TypeError):
            return np.nan  # 변환 불가능한 경우 NaN 반환

    df['odds-ratio'] = df['coef'].apply(safe_exp)
        
    return df

In [74]:
df_result = get_results(logit_func(pre_prep(df)))
df_result

Optimization terminated successfully.
         Current function value: 0.467152
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.404493
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.186207
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.386204
         Iterations 7


,coef,p-value,odds-ratio
low_const,3.6992,0.000,40.414959
low_Mean_Spend_Ratio,-3.1532,0.000,0.042715
low-mid_const,-2.2111,0.000,0.109580
low-mid_Mean_Spend_Ratio,0.5770,0.000,1.780688
mid_const,-3.9946,0.000,0.018415
mid_Mean_Spend_Ratio,1.3496,0.000,3.855883
high_const,-3.8289,0.000,0.021734
high_Mean_Spend_Ratio,2.7194,0.000,15.171217


In [75]:
# 전체 5/3 ~ 10/22
df_result = get_results(logit_func(pre_prep(df)))
df_result

Optimization terminated successfully.
         Current function value: 0.467152
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.404493
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.186207
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.386204
         Iterations 7


,coef,p-value,odds-ratio
low_const,3.6992,0.000,40.414959
low_Mean_Spend_Ratio,-3.1532,0.000,0.042715
low-mid_const,-2.2111,0.000,0.109580
low-mid_Mean_Spend_Ratio,0.5770,0.000,1.780688
mid_const,-3.9946,0.000,0.018415
mid_Mean_Spend_Ratio,1.3496,0.000,3.855883
high_const,-3.8289,0.000,0.021734
high_Mean_Spend_Ratio,2.7194,0.000,15.171217


In [76]:
df = df[(df['Mean_Spend_Ratio']>0.0)]
len(df)
df_result = get_results(logit_func(pre_prep(df)))
df_result
#소비 이력 없는 유저 제외

C:\Users\Owner\AppData\Local\Temp\ipykernel_10568\505675994.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['low'] = df[['0-1000', '1000-2999', '3000-4999', '5000-6999','7000-8999', '9000-10999', ]].sum(axis=1)
C:\Users\Owner\AppData\Local\Temp\ipykernel_10568\505675994.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['low-mid'] = df[['11000 - 12999', '13000 - 14999', '15000 - 16999', '17000 - 18999', '19000 - 20999']].sum(axis=1)
C:\Users\Owner\AppData\Local\Temp\ipykernel_10568\505675994.py:4

Optimization terminated successfully.
         Current function value: 0.488692
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.417896
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.194043
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.402756
         Iterations 7


,coef,p-value,odds-ratio
low_const,3.5682,0.000,35.452721
low_Mean_Spend_Ratio,-3.0023,0.000,0.049673
low-mid_const,-1.8792,0.000,0.152712
low-mid_Mean_Spend_Ratio,0.1727,0.000,1.188509
mid_const,-3.7788,0.000,0.022850
mid_Mean_Spend_Ratio,1.0957,0.000,2.991276
high_const,-3.7263,0.000,0.024082
high_Mean_Spend_Ratio,2.6016,0.000,13.485297


### 엔트로피 구간 별 통계

In [77]:
df

,User_ID,Mean_Spend_Ratio,Interval_Days,entropy,0-1000,1000-2999,3000-4999,5000-6999,7000-8999,9000-10999,...,31000 - 32999,33000 - 34999,35000 - 36999,37000 - 38999,39000 - 40999,41000 -,low,low-mid,mid,high
0,39867749-164c-43cc-8433-4cc0daa17956,0.875367,11.00,0.00,0,2,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0
2,1fe76913-af99-4177-98d4-1f6b0bef1dbf,0.694106,17.75,1.38,0,0,0,4,2,1,...,0,0,0,0,0,0,7,0,0,0
3,23cc23d1-e38a-4d11-854d-e005a88bd41f,1.000000,72.50,1.00,0,0,1,1,0,0,...,0,0,0,0,0,0,2,0,0,0
4,060764f0-51cf-468f-9029-84729dca69e8,0.890869,0.00,0.00,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
5,19570a6b-efd9-4273-8466-fa1192a32c89,0.638370,1.50,0.00,0,0,3,0,0,0,...,0,0,0,0,0,0,3,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163933,1059813e-9bd6-45ae-ba23-572bd604a39c,0.973611,0.00,0.00,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
163934,ff7556e4-88e7-4e2c-8247-f3084bc7d5f4,0.933672,0.00,0.00,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
163935,d6103d4a-bd38-44e5-9ca1-bff3fc628fb3,0.451592,0.00,0.00,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
163936,84cfd59a-0b64-4b01-87e4-2c024d2b648b,0.626040,0.00,0.00,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0


In [87]:
df.entropy.min(), df.entropy.max(), df.Mean_Spend_Ratio.min(), df.Mean_Spend_Ratio.max()

(0.0, 3.23, 0.0015611827520529552, 1.0)

In [129]:
def entropy_pv(df):
    # entropy 구간 설정
    bins = [0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2, 2.4, 2.6, 2.8, 3.0, 3.2, 3.4]  # 구간 설정
    labels = ['0-0.2', '0.2-0.4', '0.4-0.6', '0.6-0.8', '0.8-1.0', '1.0-1.2', 
            '1.2-1.4', '1.4-1.6', '1.6-1.8', '1.8-2.0', '2.0-2.2', '2.2-2.4',
            '2.4-2.6','2.6-2.8','2.8-3.0','3.0-3.2','3.2-3.4']  # 구간 레이블
    df['entropy_bins'] = pd.cut(df['entropy'], bins=bins, labels=labels, right=False)
    
    # 각 구간별 유저 수와 구매 횟수 집계
    result = df.groupby('entropy_bins').agg(
        user_count=('User_ID', 'nunique'),  # 유저 수
        Mean_Spend_Ratio=('Mean_Spend_Ratio','mean'),
        Interval_Days=('Interval_Days','mean'),
        low=('low', 'sum'),  # 총 구매 횟수
        low_mid=('low-mid', 'sum'),  # 총 구매 횟수
        mid=('mid', 'sum'),  # 총 구매 횟수
        high=('high', 'sum'),  # 총 구매 횟수
    ).reset_index()

    # 결과 출력
    return pd.DataFrame(result)

entropy_pv(df)

C:\Users\Owner\AppData\Local\Temp\ipykernel_10568\4038178648.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['entropy_bins'] = pd.cut(df['entropy'], bins=bins, labels=labels, right=False)
C:\Users\Owner\AppData\Local\Temp\ipykernel_10568\4038178648.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  result = df.groupby('entropy_bins').agg(


,entropy_bins,user_count,Mean_Spend_Ratio,Interval_Days,low,low_mid,mid,high
0,0-0.2,93359,0.743405,4.824369,124675,10823,4217,16982
1,0.2-0.4,134,0.546698,12.343057,2204,5,1,22
2,0.4-0.6,1112,0.577335,18.559463,9771,53,9,29
3,0.6-0.8,2227,0.599471,22.396495,13120,228,34,78
4,0.8-1.0,14301,0.644952,23.523977,50067,3004,491,925
5,1.0-1.2,20516,0.684742,21.292003,36448,6089,1615,3246
6,1.2-1.4,1768,0.577453,23.222124,10633,396,65,116
7,1.4-1.6,8723,0.637908,25.644822,29179,3692,678,995
8,1.6-1.8,378,0.533755,20.822334,2718,140,27,40
9,1.8-2.0,1004,0.572844,24.296755,5349,534,69,107


In [127]:
df__ = df[~df['Mean_Spend_Ratio'].isna()]
df__['spend_cnt'] = df__.iloc[:,-6:-2].sum(axis=1)

# entropy 구간 설정
bins = [0, 0.1, 0.2, 0.3, 0.4, 0.5,0.6,0.7,0.8,0.9,1.0]  # 구간 설정
labels = [
    '0-0.1', '0.1-0.2', '0.2-0.3', '0.3-0.4', '0.4-0.5',
    '0.5-0.6', '0.6-0.7', '0.7-0.8', '0.8-0.9', '0.9-1.0'
    ]  # 구간 레이블
df__['spend_power_bins'] = pd.cut(df__['Mean_Spend_Ratio'], bins=bins, labels=labels, right=True)

# 각 구간별 유저 수와 구매 횟수 집계
result = df__.groupby('spend_power_bins').agg(
    user_count=('User_ID', 'nunique'),  # 유저 수
    entropy=('entropy','mean'),
    Interval_Days=('Interval_Days','mean'),
    total_counts=('spend_cnt','sum'),
    low=('low', 'sum'),  # 총 구매 횟수
    low_mid=('low-mid', 'sum'),  # 총 구매 횟수
    mid=('mid', 'sum'),  # 총 구매 횟수
    high=('high', 'sum'),  # 총 구매 횟수
).reset_index()

# 결과 출력
pd.DataFrame(result)

C:\Users\Owner\AppData\Local\Temp\ipykernel_10568\1988619652.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  result = df__.groupby('spend_power_bins').agg(


,spend_power_bins,user_count,entropy,Interval_Days,total_counts,low,low_mid,mid,high
0,0-0.1,2545,0.267796,9.096764,5876,5785,69,7,34
1,0.1-0.2,5927,0.377442,10.993121,16249,15822,341,43,78
2,0.2-0.3,6814,0.484538,12.603553,20122,18845,1045,137,231
3,0.3-0.4,8207,0.562041,13.974242,25111,22892,1673,345,498
4,0.4-0.5,10140,0.625748,15.293377,29507,26303,2249,541,1254
5,0.5-0.6,11518,0.551206,14.053452,32491,28866,2591,614,1387
6,0.6-0.7,13526,0.494806,14.254714,36564,32286,3013,717,1704
7,0.7-0.8,16296,0.403213,13.611379,39724,34809,3330,833,2294
8,0.8-0.9,20321,0.345520,12.071779,42754,36599,3737,1181,3548
9,0.9-1.0,49231,0.248970,7.160654,81473,66115,7690,2951,11729


In [94]:
a = df[(df['Mean_Spend_Ratio']<=0.0) & (df['Mean_Spend_Ratio']<=0.1)].User_ID.nunique()
a += df[(df['Mean_Spend_Ratio']>0.1) & (df['Mean_Spend_Ratio']<=0.2)].User_ID.nunique()
a += df[(df['Mean_Spend_Ratio']>0.2) & (df['Mean_Spend_Ratio']<=0.3)].User_ID.nunique()
a += df[(df['Mean_Spend_Ratio']>0.3) & (df['Mean_Spend_Ratio']<=0.4)].User_ID.nunique()
a += df[(df['Mean_Spend_Ratio']>0.4) & (df['Mean_Spend_Ratio']<=0.5)].User_ID.nunique()
a += df[(df['Mean_Spend_Ratio']>0.5) & (df['Mean_Spend_Ratio']<=0.6)].User_ID.nunique()
a += df[(df['Mean_Spend_Ratio']>0.6) & (df['Mean_Spend_Ratio']<=0.7)].User_ID.nunique()
a += df[(df['Mean_Spend_Ratio']>0.7) & (df['Mean_Spend_Ratio']<=0.8)].User_ID.nunique()
a += df[(df['Mean_Spend_Ratio']>0.8) & (df['Mean_Spend_Ratio']<=0.9)].User_ID.nunique()
a += df[(df['Mean_Spend_Ratio']>0.9) & (df['Mean_Spend_Ratio']<=1.0)].User_ID.nunique()
print(a)

141980


In [95]:
df[(df['Mean_Spend_Ratio']<1.0)].User_ID.nunique()

132700

In [96]:
df[(df['Mean_Spend_Ratio'].isna())].User_ID.nunique()

0

In [97]:
df.Mean_Spend_Ratio.value_counts(dropna=False).values.sum()

144525

In [98]:
df.User_ID.nunique()

144525

In [101]:
df[(df['Mean_Spend_Ratio']==0.0)].User_ID.nunique()

0

In [42]:
df[(df['Mean_Spend_Ratio']>0.0)].User_ID.nunique()

144165

In [100]:
df.query("Mean_Spend_Ratio == 0.0")

,User_ID,Mean_Spend_Ratio,Interval_Days,entropy,0-1000,1000-2999,3000-4999,5000-6999,7000-8999,9000-10999,...,33000 - 34999,35000 - 36999,37000 - 38999,39000 - 40999,41000 -,low,low-mid,mid,high,entropy_bins
